In [29]:
from dotenv import load_dotenv

_ = load_dotenv()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage, ToolMessage


In [30]:
memory = SqliteSaver.from_conn_string(":memory:")


from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


from pydantic import BaseModel, Field
class Queries(BaseModel):
    queries: List[str]
    

    
from tavily import TavilyClient
import os
#tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [31]:
from typing import List, Optional, Dict
from langchain_core.output_parsers import JsonOutputParser

class UserPreferences(BaseModel):
    # Basic preferences
    budget: str = Field(description="Budget range the user is able to spend (e.g., low, medium, high, extra-high)")
    trip_duration: int = Field(description="Total number of days for the trip")
    destination: str = Field(description="Primary destination or country of the trip")
    
    # Travel type and group size
    trip_type: str = Field(description="Type of trip (e.g., family trip, solo travel, couple's getaway, group travel)")
    group_size: Optional[int] = Field(description="Number of people traveling together")
    
    # Interests and activities
    interests: List[str] = Field(description="List of activities the user is interested in (e.g., museums, nature, food, nightlife, shopping)")
    
    # Preferences for locations and experiences
    accommodation_type: Optional[str] = Field(description="Preferred type of accommodation (e.g., hotel, Airbnb, hostel, resort)", default="hotel")
    preferred_travel_distance: Optional[str] = Field(description="Preferred travel distance between locations (e.g., short, medium, long)", default="medium")
    
    # Meal preferences and dietary restrictions
    meal_preferences: Optional[List[str]] = Field(description="Preferences for meals (e.g., vegetarian, vegan, local cuisine, fine dining)", default=[])
    
    # Accessibility and special requirements
    accessibility_needs: Optional[bool] = Field(description="Indicates if there are any accessibility needs (e.g., wheelchair access)", default=False)
    pet_friendly: Optional[bool] = Field(description="Indicates if pet-friendly places are preferred", default=False)
    
    # Time and season preferences
    season: Optional[str] = Field(description="Preferred season for travel (e.g., summer, winter, spring, autumn)", default="any")
    flexibility: Optional[bool] = Field(description="Indicates if the user is flexible with dates or destinations", default=True)
    
    other: Optional[List[str]] = Field(description="Any other preferences the user has", default=[])

    class Config:
        schema_extra = {
            "example": {
                "budget": "medium",
                "trip_duration": 7,
                "destination": "Italy",
                "trip_type": "family trip",
                "group_size": 4,
                "interests": ["museums", "nature", "food", "beach"],
                "accommodation_type": "resort",
                "preferred_travel_distance": "short",
                "meal_preferences": ["local cuisine", "vegetarian"],
                "accessibility_needs": False,
                "pet_friendly": False,
                "season": "summer",
                "flexibility": True,
                "other": []
            }
        }


user_preferences_parser = JsonOutputParser(pydantic_object=UserPreferences)

c:\Users\logis\AppData\Local\pypoetry\Cache\virtualenvs\ai-travel-planner-WLuryUIH-py3.11\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [32]:
# Define the schema for information about a city
class CityInfo(BaseModel):
    destinations: List[str] = Field(default=[], description="List of tourist destinations or attractions in the city")
    restaurants: List[str] = Field(default=[], description="List of recommended restaurants/cafes in the city")
    accomodation: List[str] = Field(default=[], description="List of suggested accomodation in the city")
    activities: Optional[List[str]] = Field(default=[], description="Popular activities or experiences to do in the city")
    events: Optional[List[str]] = Field(default=[], description="Current or upcoming events in the city")
    #shopping: Optional[List[str]] = Field(default=[], description="Recommended shopping areas or stores in the city")
    #nightlife: Optional[List[str]] = Field(default=[], description="Nightlife spots, clubs, or entertainment venues")
    #parks_nature_spots: Optional[List[str]] = Field(default=[], description="Parks, gardens, or nature spots in or near the city")
    #transportation: Optional[List[str]] = Field(default=[], description="Tips and recommendations for local transportation")
    #cultural_experiences: Optional[List[str]] = Field(default=[], description="Specific cultural experiences or activities")
    #local_tips: Optional[List[str]] = Field(default=[], description="Insider tips or local recommendations")
    #health_safety: Optional[List[str]] = Field(default=[], description="Health and safety information in the city")
    #weather_seasonal_info: Optional[List[str]] = Field(default=[], description="Average weather conditions or seasonal information")
    #day_trips: Optional[List[str]] = Field(default=[], description="Recommended day trips or nearby excursions from the city")
    #family_friendly: Optional[List[str]] = Field(default=[], description="Activities or attractions suitable for families with children")
    #pet_friendly_places: Optional[List[str]] = Field(default=[], description="Pet-friendly places such as parks, hotels, or restaurants")
    #photo_spots: Optional[List[str]] = Field(default=[], description="Suggested locations for taking memorable photos or enjoying scenic views")
    #accessibility: Optional[List[str]] = Field(default=[], description="Accessibility information for destinations, public transport, and hotels")

# Define the schema for the entire ideas generation structure
class TravelIdeas(BaseModel):
    ideas: Dict[str, CityInfo] = Field(description="Dictionary containing city names as keys and their respective information as values")
    
    class Config:
        schema_extra = {
            "example": {
                "ideas": {
                    "Rome, Italy": {
                        "destinations": ["Colosseum", "Vatican City", "Roman Forum", "Pantheon"],
                        "restaurants": ["Ristorante Aroma", "La Pergola", "Felice a Testaccio"],
                        "hotels": ["Hotel Hassler", "Hotel Eden", "The St. Regis Rome"],
                        "activities": ["Gondola ride", "Guided tour of Roman ruins", "Nighttime Colosseum tour"],
                        "shopping": ["Via Condotti", "Campo de' Fiori market"],
                        #"nightlife": ["Harry's Bar", "Live jazz at Gregory's Jazz Club"],
                        #"cultural_experiences": ["Cooking class to learn Italian cuisine"],
                        #"photo_spots": ["Gianicolo Hill", "Piazza Navona"],
                        #"parks_nature_spots": ["Villa Borghese", "Giardino degli Aranci"],
                        #"transportation": ["Best way to get around Rome is by metro", "Use public buses for sightseeing"],
                        #"local_tips": ["Avoid visiting the Colosseum on weekends", "Try local street food at Testaccio Market"],
                        #"health_safety": ["Nearest hospital: Policlinico Umberto I", "Emergency contact numbers: 112"],
                        #"weather_seasonal_info": ["Best time to visit Rome is from April to June", "Expect high temperatures in August"],
                        #"day_trips": ["Day trip to Pompeii", "Visit Tivoli for Villa d'Este"],
                        #"family_friendly": ["Visit the Children's Museum of Rome", "Explora Children's Museum"],
                        #"pet_friendly_places": ["Pet-friendly hotel: Hotel Santa Maria", "Dog park: Parco Savello"],
                        #"accessibility": ["Colosseum has an elevator for wheelchair access", "Accessible public transport routes available"],
                    },
                    "Florence, Italy": {
                        "destinations": ["Florence Cathedral", "Uffizi Gallery", "Ponte Vecchio", "Boboli Gardens"],
                        "restaurants": ["Osteria Francescana", "Enoteca Pinchiorri", "Trattoria Mario"],
                        "hotels": ["Hotel Savoy", "Four Seasons Hotel Firenze", "Villa Cora"],
                        "activities": ["Art tour of the Uffizi Gallery", "Visit to the Accademia Gallery"],
                        #"shopping": ["Via de' Tornabuoni", "Mercato Centrale"],
                        #"cultural_experiences": ["Florence Duomo climbing experience", "Cooking class in Florence"],
                        #"photo_spots": ["Piazzale Michelangelo", "View from Ponte Vecchio"],
                        #"parks_nature_spots": ["Boboli Gardens", "Giardino Bardini"]
                    }
                }
            }
        }

travel_ideas_parser = JsonOutputParser(pydantic_object=TravelIdeas)


In [33]:
class AgentState(TypedDict):
    user_query: str
    user_preferences: UserPreferences
    travel_ideas: TravelIdeas
    itinerary: str
    messages: Annotated[List[AnyMessage], operator.add]

In [34]:
from langchain_core.prompts import PromptTemplate

PREFERENCES_EXTRACTION_PROMPT = PromptTemplate(
    template = """
    You are an expert travel agent tasked with extracting structured information to fill out a questionnaire about the customer's planned trip.
    Based on the customer's query, extract the key information about their trip preferences and format it accordingly.

    Provide the extracted preferences in the following structured format:
    {format_instructions}

    Customer's Query:
    {user_query}
    """,
    input_variables=["user_query"],
    partial_variables={"format_instructions": user_preferences_parser.get_format_instructions()},
)


IDEAS_GENERATION_PROMPT = PromptTemplate(
    template = """
    You are an expert travel agent tasked with creating a list of places the customer may want to visit.
    Based on the customer's preferences provided below, suggest a list of diverse places that fit the customer's interests. 
    Ensure the list includes a mix of well-known and hidden gem destinations, while also considering the geographic proximity of locations to make travel between them feasible. 
    Provide more ideas than the customer can visit within the trip, as they will later decide what to include and exclude from their itinerary.
    
    Provide the travel ideas in the following structured format:
    {format_instructions}

    Customer preferences:
    {user_preferences}
    """,
    input_variables=['user_preferences'],
    partial_variables={"format_instructions": travel_ideas_parser.get_format_instructions()},
)


ITINERARY_PLANNING_PROMPT = """
    You are an expert travel agent tasked with creating a detailed trip itinerary based on the customer's preferences and a dictionary of suggested places to visit in different cities. 
    Please create an itinerary that balances key attractions and activities with the practical aspects of travel, such as distances, travel time, and meal or rest breaks. 
    Provide a structured plan, distributing activities across each day of the trip.
    For each major activity try to offer at least two alternatives so the customer may choose between them.

    Consider the following factors:
    - Prioritize activities that match the customer's preferences.
    - Ensure travel between locations is feasible, taking into account distances and available time.
    - Provide suggestions for meals or breaks at suitable times.
    - Offer a diverse range of activities to allow flexibility in planning.

    Use the search engine to look up information. \
    You are allowed to make multiple calls (either together or in sequence). \
    Only look up information when you are sure of what you want. \
    If you need to look up some information before asking a follow up question, you are allowed to do that!

    Utilize all the information below as needed:

    ------
    Customer preferences:
    {user_preferences}

    ------
    Inspiration ideas:
    {travel_ideas}
    """


In [35]:
def preferences_extraction_node(state: AgentState):
    chain = PREFERENCES_EXTRACTION_PROMPT | model | user_preferences_parser
    response = chain.invoke({"user_query": state['user_query']})
    return {"user_preferences": response}


def ideas_generation_node(state: AgentState):
    chain = IDEAS_GENERATION_PROMPT | model | travel_ideas_parser
    response = chain.invoke({"user_preferences": state['user_preferences']})
    return {"travel_ideas": response}

def itinerary_planning_node(state: AgentState):
    messages = [
        SystemMessage(
            content=ITINERARY_PLANNING_PROMPT.format(travel_ideas=state['travel_ideas'], user_preferences=state['user_preferences'])
        )
        ]
    response = model.invoke(messages)
    return {"itinerary": response.content}

'''
def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

def take_action(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
        print(f"Calling: {t}")
        result = self.tools[t['name']].invoke(t['args'])
        results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
    print("Back to the model!")
    return {'messages': results}
'''

builder = StateGraph(AgentState)
builder.add_node("preferences_extraction", preferences_extraction_node)
builder.add_node("ideas_generation", ideas_generation_node)
builder.add_node("itinerary_planning", itinerary_planning_node)
builder.set_entry_point("preferences_extraction")
builder.add_edge("preferences_extraction", "ideas_generation")
builder.add_edge("ideas_generation", "itinerary_planning")
graph = builder.compile(checkpointer=memory)

In [36]:
user_task = """
    I am planning a 3-day family trip to Italy. We enjoy historical sites, good food, and outdoor activities. \
    We’d like to visit different cities and explore famous landmarks, but also have some relaxing days in nature. \
    Would like to keep one day without any activities, just to say at the hotel and rest.\
    Our budget is moderate, and we prefer shorter travel distances between destinations.
    """



with SqliteSaver.from_conn_string(":memory:") as memory:
    graph = builder.compile(checkpointer=memory)
    thread = {"configurable": {"thread_id": "1"}}
    response = []

    for s in graph.stream({
        'user_query': user_task,
    }, thread):
        print(s)
        response.append(s)

{'preferences_extraction': {'user_preferences': {'budget': 'moderate', 'trip_duration': 3, 'destination': 'Italy', 'trip_type': 'family trip', 'group_size': None, 'interests': ['historical sites', 'good food', 'outdoor activities'], 'accommodation_type': 'hotel', 'preferred_travel_distance': 'short', 'meal_preferences': [], 'accessibility_needs': False, 'pet_friendly': None, 'season': 'any', 'flexibility': True, 'other': ['one day without activities']}}}
{'ideas_generation': {'travel_ideas': {'ideas': {'Rome': {'destinations': ['Colosseum', 'Roman Forum', 'Trevi Fountain', 'Pantheon'], 'restaurants': ['La Pergola', 'Roscioli', 'Da Enzo al 29'], 'accomodation': ['Hotel Eden', 'Hotel Artemide', 'Hotel Raphael'], 'activities': ['Visit Vatican City', 'Explore Trastevere neighborhood', 'Cooking class to learn Italian cuisine'], 'events': ['Outdoor concerts in Villa Borghese Gardens', 'Local food festivals']}, 'Florence': {'destinations': ['Duomo Cathedral', 'Uffizi Gallery', 'Ponte Vecchio'

In [38]:
from IPython.display import Markdown, display

response_dict = {}
for node_response in response:
    response_dict.update(node_response)

# Itinerary text with markdown formatting
preferences_json = response_dict['preferences_extraction']['user_preferences']
ideas_json = response_dict['ideas_generation']['travel_ideas']
itinerary_md = response_dict['itinerary_planning']['itinerary']


# Display the markdown
display(preferences_json)
display(ideas_json)
display(Markdown(itinerary_md))


{'budget': 'moderate',
 'trip_duration': 3,
 'destination': 'Italy',
 'trip_type': 'family trip',
 'group_size': None,
 'interests': ['historical sites', 'good food', 'outdoor activities'],
 'accommodation_type': 'hotel',
 'preferred_travel_distance': 'short',
 'meal_preferences': [],
 'accessibility_needs': False,
 'pet_friendly': None,
 'season': 'any',
 'flexibility': True,
 'other': ['one day without activities']}

{'ideas': {'Rome': {'destinations': ['Colosseum',
    'Roman Forum',
    'Trevi Fountain',
    'Pantheon'],
   'restaurants': ['La Pergola', 'Roscioli', 'Da Enzo al 29'],
   'accomodation': ['Hotel Eden', 'Hotel Artemide', 'Hotel Raphael'],
   'activities': ['Visit Vatican City',
    'Explore Trastevere neighborhood',
    'Cooking class to learn Italian cuisine'],
   'events': ['Outdoor concerts in Villa Borghese Gardens',
    'Local food festivals']},
  'Florence': {'destinations': ['Duomo Cathedral',
    'Uffizi Gallery',
    'Ponte Vecchio',
    'Boboli Gardens'],
   'restaurants': ['Trattoria Mario',
    'La Giostra',
    'Osteria Cinghiale Bianco'],
   'accomodation': ['Hotel Lungarno', 'Hotel Savoy', 'Portrait Firenze'],
   'activities': ["Visit the Accademia Gallery to see Michelangelo's David",
    'Walk up to Piazzale Michelangelo for panoramic views',
    'Bike tour in the Tuscan countryside'],
   'events': ["Opera performances at St. Mark's Anglican Church",
    'Artisan mar

Based on your preferences and the suggested destinations and activities in Italy, here is a detailed itinerary for your 3-day family trip:

---

### Day 1: Rome
- **Morning:**
  - Option 1: Visit the **Colosseum** and **Roman Forum**
  - Option 2: Explore the **Trastevere neighborhood**
- **Lunch:** Enjoy a meal at **Roscioli** or **Da Enzo al 29**
- **Afternoon:**
  - Option 1: Visit the **Trevi Fountain** and **Pantheon**
  - Option 2: Take a **cooking class** to learn Italian cuisine
- **Dinner:** Taste exquisite dishes at **La Pergola**
- **Evening:** Attend an **outdoor concert** in Villa Borghese Gardens

### Day 2: Florence
- **Morning:**
  - Option 1: Visit the **Duomo Cathedral** and **Uffizi Gallery**
  - Option 2: Explore the **Boboli Gardens**
- **Lunch:** Try local cuisine at **Trattoria Mario** or **Osteria Cinghiale Bianco**
- **Afternoon:**
  - Option 1: Visit the **Ponte Vecchio** and **Accademia Gallery** to see Michelangelo's David
  - Option 2: Walk up to **Piazzale Michelangelo** for panoramic views
- **Dinner:** Dine at **La Giostra**
- **Evening:** Enjoy an **opera performance** at St. Mark's Anglican Church

### Day 3: Amalfi Coast
- **Morning:**
  - Option 1: Explore **Positano** and relax on its beaches
  - Option 2: Take a **boat tour** along the coast
- **Lunch:** Indulge in a meal at **Da Adolfo** or **Ristorante Saraceno d'Oro**
- **Afternoon:**
  - Option 1: Hike the **Path of the Gods**
  - Option 2: Visit **Ravello** and attend a concert at Villa Rufolo
- **Dinner:** Savor a delightful dinner at **Le Sirenuse**
- **Evening:** Enjoy the serene atmosphere of the Amalfi Coast

### Additional Notes:
- **Accommodation:** You can stay at **Hotel Eden** in Rome, **Hotel Lungarno** in Florence, and **Belmond Hotel Caruso** on the Amalfi Coast.
- **Transportation:** Consider using trains or private transfers for travel between cities to maximize your time.
- **Meals:** Feel free to explore local eateries and try different cuisines based on your preferences.
- **Rest Day:** You can plan a relaxed day without activities in any of the destinations to unwind and recharge.

---

This itinerary offers a mix of historical sites, good food, and outdoor activities while ensuring a balanced and enjoyable experience for your family trip to Italy. Feel free to adjust any activities based on your interests and preferences. Enjoy your trip!